# Movie Recommendation Predict

![image info](https://user-images.githubusercontent.com/33485020/108069438-5ee79d80-7089-11eb-8264-08fdda7e0d11.jpg)

### Honour Code

Idongesit Bokeime, Ngawetu Mtirara, Daniel Uwaoma, Sibusiso Sibiya, Nonokazi Cele, Mbuyiselo Mkwanazi, Collins Kgooho and Pharis Ihaki (Team_CW3), confirm - by submitting this document - that the solutions in this notebook are a result of my own work and that I abide by the [EDSA honour code](https://drive.google.com/file/d/1QDCjGZJ8-FmJE3bZdIQNwnJyQKPhHZBn/view?usp=sharing).

Non-compliance with the honour code constitutes a material breach of contract.


### Problem Statement

In today’s technology driven world, recommender systems are socially and economically critical for ensuring that individuals can make appropriate choices surrounding the content they engage with on a daily basis. One application where this is especially true surrounds movie content recommendations; where intelligent algorithms can help viewers find great titles from tens of thousands of options.

With this context, EDSA is challenging you to construct a recommendation algorithm based on content or collaborative filtering, capable of accurately predicting how a user will rate a movie they have not yet viewed based on their historical preferences.

Providing an accurate and robust solution to this challenge has immense economic potential, with users of the system being exposed to content they would like to view or purchase - generating revenue and platform affinity. (To be fixed, copied from kaggle)

### Objectives

1.   Explore and visualize the dataset.
2.   Clean and engineer the dataset.
3.   Build multiple models that will analyse sentiments on climate change      make the necesssary predictions.
4.   Assess the accuracy of the models.
5.   Choose the best model to make predictions.

# Table of contents:
- [1. Import libraries and datasets](#section-one)
  * [1.1. Loading the data](#subsection-one)
- [2. Dataset at a glance](#section-two)
- [3. Preprocessing](#section-three)
  * [3.1. Add new column](#subsection-one)
  * [3.2. Hashtag extraction](#subsection-two)
  * [3.3. Cleaning the corpus](#subsection-three)
  * [3.4. Parts of speech tagging and lemmatization](#subsection-four)
  * [3.5. Word frequency](#subsection-five)
  * [3.6. Named entity extraction](#subsection-six)
- [4. Exploratory data analysis](#section-four)
  * [4.1. Target variable distribution](#subsection-one)
  * [4.2. Tweet length distribution](#subsection-two)
  * [4.3. Climate change buzzwords](#subsection-three)
  * [4.4. Hashtags](#subsection-four)
  * [4.5. Named Entity Recognition](#subsection-five)
- [5. Model building](#section-five)
  * [5.1. Train-validation split](#subsection-one)
  * [5.2. Pipelines](#subsection-three)
  * [5.3. Model training](#subsection-four)
- [6. Model evaluation](#section-six)
  * [6.1. Logging on Comet](#subsection-one)
  * [6.2. Hyperparameter Tuning](#ubsection-two)
- [7. Conclusion](#section-seven)
- [8. Submission](#section-eight)
- [9. References](#section-nine)

 <a id="two"></a>
## 1. Importing Packages

<a id="three"></a>
### 1.1. Loading the Data
<a class="anchor" id="1.1">

<a id="section-two"></a>
## 2. Dataset at a glance

<a id="section-two"></a>
## 3. Exploratory data analysis

<a id="section-two"></a>
## 4. Feature Engineering

<a id="section-two"></a>
## 5. Recommender Systems

<a id="three"></a>
### 5.1. Content Filtering-Based Recommender System
<a class="anchor" id="1.1">

<a id="three"></a>
### 5.2. Collaborative Filtering-Based Recommender System
<a class="anchor" id="1.1">

<a id="section-six"></a>
## 6. Model evaluation

<a id="section-seven"></a>
## 7. Model Selection 

<a id="section-seven"></a>
## 8. Conclusion

<a id="section-eight"></a>
## 9. Submission

<a id="three"></a>
### 9.1. Creating pickle files of the models
<a class="anchor" id="1.1">

<a id="three"></a>
### 9.2.  Creating kaggle submission file
<a class="anchor" id="1.1">

<a id="section-nine"></a>
## 10. References 